In [ ]:
import polars as pl

behaviors = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/test/behaviors.parquet')
history = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/test/history.parquet')
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/articles.parquet')

In [ ]:
behaviors.head()

In [ ]:
from polimi.utils._catboost import _preprocessing_article_endorsement_feature

articles_endorsement = _preprocessing_article_endorsement_feature(behaviors=behaviors, period="10h")
articles_endorsement.head()

In [ ]:
behaviors.head()

In [ ]:
behaviors.filter(pl.col('impression_id') == 0).shape[0]

In [ ]:
from polimi.utils._catboost import _preprocessing_history_trendiness_scores

users_mean_trendiness_scores, topics_mean_trendiness_scores = _preprocessing_history_trendiness_scores(
        history=history, articles=articles)

In [ ]:
users_mean_trendiness_scores.head()

In [ ]:
topics_mean_trendiness_scores.head()

In [ ]:
behaviors.head()

In [ ]:
behaviors.filter(pl.col('impression_id') == 0).shape[0]

In [ ]:
from polimi.utils._catboost import _preprocessing_mean_delay_features, _preprocessing_window_features

topic_mean_delays, user_mean_delays = _preprocessing_mean_delay_features(
        articles=articles, history=history)

windows, user_windows, user_topics_windows, user_category_windows = _preprocessing_window_features(
        history=history, articles=articles)

In [ ]:
from polimi.utils._catboost import get_unique_categories

unique_categories = get_unique_categories(articles)

In [ ]:
user_category_windows.head()

In [ ]:
slices_gen = behaviors.iter_slices(20000)

In [ ]:
first_behaviors = next(slices_gen)

In [ ]:
first_behaviors.head()

In [ ]:
first_behaviors = first_behaviors.explode('article_ids_inview').rename({'article_ids_inview': 'article'})
first_behaviors.head()

In [ ]:
from polimi.utils._catboost import add_article_endorsement_feature

end = add_article_endorsement_feature(first_behaviors, articles_endorsement)
end.head()

In [ ]:
from polimi.utils._catboost import add_history_trendiness_scores_feature

topics = articles.select("topics").explode("topics").unique()
topics = [topic for topic in topics["topics"] if topic is not None]

tmp = add_history_trendiness_scores_feature(
    first_behaviors, articles=articles, users_mean_trendiness_scores=users_mean_trendiness_scores,
    topics_mean_trendiness_scores=topics_mean_trendiness_scores, topics=topics)
tmp.head()

In [ ]:
from polimi.utils._catboost import add_mean_delays_features

tmp = add_mean_delays_features(first_behaviors, articles=articles, topic_mean_delays=topic_mean_delays,
                               user_mean_delays=user_mean_delays)
tmp.head()

In [ ]:
first_behaviors = first_behaviors.join(articles.select(['article_id', 'category']), left_on='article', right_on='article_id', how='left')
first_behaviors.head()

In [ ]:
from polimi.utils._catboost import add_window_features

tmp = add_window_features(first_behaviors, articles=articles, user_windows=user_windows,
                          user_category_windows=user_category_windows, user_topics_windows=user_topics_windows, windows=windows)
tmp.head()

In [ ]:
from polimi.utils._catboost import add_trendiness_feature_categories

tmp = add_trendiness_feature_categories(first_behaviors, articles=articles)
tmp.head()

In [ ]:
from polimi.utils._topic_model import _compute_topic_model, add_topic_model_features

articles, topic_model_columns, n_components = _compute_topic_model(articles)

In [ ]:
import polars as pl

tmp = pl.read_parquet('/home/ubuntu/experiments/preprocessing_test_2024-05-07_17-07-46/Sliced_ds/test_slice_2.parquet')
tmp.head()